In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
folder = 'gs://ccle_bams/'

In [ ]:
samples = os.popen('gsutil -m ls -al ' + folder + '**.bam').read().split('\n')

In [ ]:
sizes = {'gs://' + val.split('gs://')[1].split('#')[0]: int(val.split("2019-")[0]) for val in samples[:-2]}

In [ ]:
for k, val in sizes.items():
    sizes[k] = k.split('/')[-2]

In [ ]:
versions = set(sizes.values())
versions

In [ ]:
v1 = []
v2 = []
v3 = []
v4 = []
v5 = []
v6 = []
unknown = []
current= []
name = []
for k, val in sizes.items():
    name.append('k.split('/')[-1].split('.bam')[0])
    v1.append(k if val =='v1' else np.NaN)
    v2.append(k if val =='v2' else np.NaN)
    v3.append(k if val =='v3' else np.NaN)
    v4.append(k if val =='v4' else np.NaN)
    v5.append(k if val =='v5' else np.NaN)
    v6.append(k if val =='v6' else np.NaN)
    current.append(k if val =='current' else np.NaN)
    unknown.append(k if val in ['Getz_IBM_CellLines_Exomes_19samples_02012018', 'bam.analysis.ready', 'ccle.wes.gdc', 'wes', 'rna'] else np.NaN)

In [ ]:
data = pd.DataFrame({'unknown':unknown,
                    'v1':v1,
                    'v2':v2,
                    'v3':v3,
                    'v4':v4,
                    'v5':v5,
                    'v6':v6,
                    'current':current},index=name)

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.columns

In [ ]:
len(data)

In [ ]:
gb = data.groupby(('index'))

In [ ]:
res = pd.DataFrame(gb[data.columns[0]].unique())

In [ ]:
for i in data.columns[1:]:
    res[i] = gb[i].unique()

In [ ]:
for k, val in res.iterrows():
    for j, i in val.iteritems():
        put=False
        for v in i:
            if v is not np.NaN:
                res.loc[k,j] = v
                put = True
        if not put:
            res.loc[k,j] = np.NaN

In [ ]:
res = res.drop('index',1)

In [ ]:
res.to_csv('state.csv')

In [ ]:
res = pd.read_csv('ccle_versions_state.csv')

In [ ]:
for i, val in res.iterrows():
    if val.v1 is np.nan:
        if val.unknown is not np.nan:
            res.loc[i,'v1'] = val.unknown
            res.loc[i,'unknown'] = np.nan
        elif val.current is not np.nan:
            res.loc[i,'v1'] = val.current
            res.loc[i,'current'] = np.nan
            

In [ ]:
res.isna().sum(0)/len(res)

In [ ]:
res